## Evaluate F1 - score or Confusion matrix by using Fine-Tuning ResNet18

In [17]:
import os 
import time
import numpy as np

from sklearn.metrics import f1_score, confusion_matrix, classification_report
from tqdm import tqdm

import torch
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader, ConcatDataset

import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

from PIL import Image

## GPU Setting ###
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print(DEVICE)

cuda


In [18]:
class CUB_CAPTCHA(Dataset):
    def __init__(self, transform, mode='train'):
        self.transform = transform
        self.mode = mode
        base_path = 'CAPTCHA_IMAGES_GAN'  # 기본 경로 설정

        if self.mode == 'train':
            folder_path = os.path.join(base_path, 'train')
        elif self.mode == 'valid':
            folder_path = os.path.join(base_path, 'val')

        # 모든 하위 폴더의 파일을 포함
        self.image_paths = []
        self.labels = []

        for class_name in os.listdir(folder_path):
            class_path = os.path.join(folder_path, class_name)
            if os.path.isdir(class_path):
                for img_name in os.listdir(class_path):
                    img_path = os.path.join(class_path, img_name)
                    if os.path.isfile(img_path):  # 파일만 추가
                        self.image_paths.append(img_path)
                        self.labels.append(class_name)  # 클래스명으로 라벨 설정

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        img = Image.open(img_path).convert('RGB')
        img = self.transform(img)

        # 라벨을 클래스 이름에서 추출
        label = self.labels[idx]

        # 문자열 라벨을 숫자로 매핑
        label_map = {  # 클래스 이름을 정수로 매핑
            'motorcycle': 0, 'seaplane': 1, 'boat': 2, 'motorbus': 3,
            'bicycle': 4, 'train': 5, 'truck': 6, 'airplane': 7
        }
        label = label_map[label]

        return img, label


In [5]:
"""import splitfolders
splitfolders.ratio("CAPTCHA_DCGAN", output="CAPTCHA_DCGAN_split", seed=1004, ratio=(0, 1))"""

Copying files: 140 files [00:00, 207.56 files/s]


In [19]:
import os
from PIL import Image
from torch.utils.data import Dataset

class CUB_CAPTCHA_TEST(Dataset):
    def __init__(self, transform, mode='valid'):
        self.transform = transform
        self.mode = mode
        base_path = 'CAPTCHA_DCGAN_split'  # 기본 경로 설정

        # train과 validation에 따라 폴더 경로 설정
        if self.mode == 'train':
            folder_path = os.path.join(base_path, 'train')
        elif self.mode == 'valid':
            folder_path = os.path.join(base_path, 'val')
        else:
            raise ValueError("mode must be 'train' or 'valid'")

        # 모든 하위 폴더의 파일 경로와 라벨 생성
        self.image_paths = []
        self.labels = []

        # 클래스 이름을 정수로 매핑
        self.label_map = {
            'motorcycle': 0, 'seaplane': 1, 'boat': 2, 'motorbus': 3,
            'bicycle': 4, 'train': 5, 'truck': 6, 'airplane': 7
        }

        for class_name in os.listdir(folder_path):
            class_path = os.path.join(folder_path, class_name)
            if os.path.isdir(class_path) and class_name in self.label_map:  # 유효한 클래스 이름만 포함
                for img_name in os.listdir(class_path):
                    img_path = os.path.join(class_path, img_name)
                    if os.path.isfile(img_path):  # 파일만 추가
                        self.image_paths.append(img_path)
                        self.labels.append(class_name)  # 클래스명으로 라벨 설정

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        img = Image.open(img_path).convert('RGB')
        img = self.transform(img)

        # 라벨을 클래스 이름에서 추출하고 정수로 변환
        label = self.labels[idx]
        label = self.label_map[label]

        return img, label


In [20]:
# 데이터 변환 정의
'''transforms_train = transforms.Compose([
    transforms.Resize((448, 448)),              # 크기 조정
    transforms.RandomRotation(15),              # 무작위 회전
    transforms.RandomResizedCrop(448, scale=(0.8, 1.0)),  # 무작위 크롭
    transforms.RandomHorizontalFlip(p=0.5),     # 좌우 반전
    transforms.ToTensor(),                      # 텐서 변환
])'''

transforms_train = transforms.Compose([
    transforms.Resize((64, 64)),              # 크기 조정
    transforms.ToTensor(),                      # 텐서 변환
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # 픽셀 값을 [-1, 1]로 정규화
])

transforms_val = transforms.Compose([
    transforms.Resize((64, 64)),              # 크기 조정
    transforms.ToTensor(),                      # 텐서 변환
])


In [21]:
BATCH_SIZE = 16

original_train_set = CUB_CAPTCHA(mode='train', transform=transforms_train) 
original_val_set = CUB_CAPTCHA(mode='valid', transform=transforms_train)
CAPTCHA_train_set = ConcatDataset([original_train_set, original_val_set])

val_set = CUB_CAPTCHA_TEST(mode = 'valid', transform = transforms_val)

train_loader = DataLoader(CAPTCHA_train_set, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=True)

print('Num of each dataset:', len(CAPTCHA_train_set), len(val_set))
print("Loaded combined dataloader")

Num of each dataset: 4068 122
Loaded combined dataloader


## Pre - trained ResNet Load

In [22]:
## Model / Optimizer ###
EPOCH = 30
lr = 0.001

model = models.resnet18(weights = "DEFAULT")

### Transfer Learning ###
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 8)
model.to(DEVICE)

params_to_update = []
for name,param in model.named_parameters():
    if 'fc' in name:
        print(name)
        print(param.requires_grad)
        param.requires_grad = True
        params_to_update.append(param)
    else:
        param.requires_grad = False

print("Created a learning model and optimizer")

fc.weight
True
fc.bias
True
Created a learning model and optimizer


In [23]:
optimizer = optim.Adam(params_to_update, lr = 0.001)

In [27]:
### Train / Evaluation ###
def train(model, train_loader, optimizer, epoch):
    model.train()
    for i, (image,target) in enumerate (train_loader):
        image, target = image.to(DEVICE), target.to(DEVICE)
        output = model(image)
        optimizer.zero_grad()
        train_loss = F.cross_entropy(output, target).to(DEVICE)
        
        train_loss.backward() 
        optimizer.step()
        
        if i % 10 ==0:
            print(f"Train_Epoch : {epoch} [{i}/{len(train_loader)}]\tLoss: {train_loss.item():.6f}")
    
    return train_loss

def evaluate(model, val_loader):
    model.eval()
    eval_loss = 0
    correct = 0
    all_preds = []
    all_targets = []
    
    with torch.no_grad():
        for i, (image, target) in enumerate(val_loader):
            image, target = image.to(DEVICE), target.to(DEVICE)
            output = model(image)

            eval_loss += F.cross_entropy(output, target, reduction='sum').item()
            
            # Predicted labels
            pred = output.argmax(dim=1)  # 각 배치에서 가장 높은 값의 인덱스를 예측
            all_preds.extend(pred.cpu().numpy())  # GPU → CPU
            all_targets.extend(target.cpu().numpy())
            
            correct += pred.eq(target).sum().item()
    
    eval_loss /= len(val_loader.dataset)
    eval_accuracy = 100 * correct / len(val_loader.dataset)
    
    # Calculate F1-score
    f1 = f1_score(all_targets, all_preds, average='weighted')
    
    # Generate Confusion Matrix
    conf_matrix = confusion_matrix(all_targets, all_preds)

    # Extract unique classes from the dataset
    unique_classes = sorted(set(all_targets + all_preds))
    target_names = [f"Class {cls}" for cls in unique_classes]
    
    # Print classification report
    print("\nClassification Report:")
    print(classification_report(all_targets, all_preds, target_names=target_names))
    
    print("\nConfusion Matrix:")
    print(conf_matrix)
    
    return eval_loss, eval_accuracy, f1, conf_matrix



In [28]:
import time
import torch.optim as optim
from tqdm import tqdm
import torch

epochs = 30

def train_model():
    start = time.time()
    best = 0
    
    for epoch in tqdm(range(epochs), desc="Epochs"):
        epoch_start = time.time()
        train_loss = train(model, train_loader, optimizer, epoch)
        epoch_end = time.time()
            
        # 각 에포크에 대한 Computation Time, Memory 사용량 기록
        epoch_time = epoch_end - epoch_start
        
        # Test 결과 기록
        test_loss, test_accuracy, test_f1, conf_matrix = evaluate(model, val_loader)
        
        print(f"[Final] Test Loss: {test_loss:.4f}, Accuracy: {test_accuracy:.4f}%, F1-Score: {test_f1:.4f}")
        
        print("Confusion Matrix:")
        print(conf_matrix)
        
        end = time.time()
        elapsed_time = end - start
        print(f"Elapsed Time: {int(elapsed_time / 3600)}h, {int((elapsed_time % 3600) / 60)}m, {int(elapsed_time % 60)}s")

In [29]:
train_model()

Epochs:   0%|          | 0/30 [00:00<?, ?it/s]

Train_Epoch : 0 [0/255]	Loss: 0.394523
Train_Epoch : 0 [10/255]	Loss: 0.233074
Train_Epoch : 0 [20/255]	Loss: 0.311045
Train_Epoch : 0 [30/255]	Loss: 0.392620
Train_Epoch : 0 [40/255]	Loss: 0.781540
Train_Epoch : 0 [50/255]	Loss: 0.780638
Train_Epoch : 0 [60/255]	Loss: 0.419934
Train_Epoch : 0 [70/255]	Loss: 0.544697
Train_Epoch : 0 [80/255]	Loss: 0.334458
Train_Epoch : 0 [90/255]	Loss: 0.413589
Train_Epoch : 0 [100/255]	Loss: 0.346860
Train_Epoch : 0 [110/255]	Loss: 0.760581
Train_Epoch : 0 [120/255]	Loss: 0.192426
Train_Epoch : 0 [130/255]	Loss: 0.399808
Train_Epoch : 0 [140/255]	Loss: 0.224354
Train_Epoch : 0 [150/255]	Loss: 0.198025
Train_Epoch : 0 [160/255]	Loss: 0.942798
Train_Epoch : 0 [170/255]	Loss: 0.329061
Train_Epoch : 0 [180/255]	Loss: 0.325506
Train_Epoch : 0 [190/255]	Loss: 0.535804
Train_Epoch : 0 [200/255]	Loss: 0.765478
Train_Epoch : 0 [210/255]	Loss: 0.290502
Train_Epoch : 0 [220/255]	Loss: 0.334655
Train_Epoch : 0 [230/255]	Loss: 0.659995
Train_Epoch : 0 [240/255]	L

Epochs:   3%|▎         | 1/30 [00:07<03:34,  7.39s/it]


Classification Report:
              precision    recall  f1-score   support

     Class 0       1.00      0.14      0.25        28
     Class 1       0.00      0.00      0.00         1
     Class 2       0.06      0.25      0.10        12
     Class 4       0.00      0.00      0.00         1
     Class 5       1.00      0.14      0.24        22
     Class 6       0.68      0.83      0.75        18
     Class 7       0.75      0.30      0.43        40

    accuracy                           0.30       122
   macro avg       0.50      0.24      0.25       122
weighted avg       0.76      0.30      0.36       122


Confusion Matrix:
[[ 4  0  4 13  0  6  1]
 [ 0  0  0  0  0  0  1]
 [ 0  9  3  0  0  0  0]
 [ 0  0  0  0  0  0  1]
 [ 0  1 18  0  3  0  0]
 [ 0  0  2  0  0 15  1]
 [ 0  4 23  0  0  1 12]]
[Final] Test Loss: 1.8785, Accuracy: 30.3279%, F1-Score: 0.3613
Confusion Matrix:
[[ 4  0  4 13  0  6  1]
 [ 0  0  0  0  0  0  1]
 [ 0  9  3  0  0  0  0]
 [ 0  0  0  0  0  0  1]
 [ 0  1 18  0

/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Epochs:   7%|▋         | 2/30 [00:14<03:25,  7.32s/it]

...


Classification Report:
              precision    recall  f1-score   support

     Class 0       1.00      0.71      0.83        28
     Class 1       0.00      0.00      0.00         1
     Class 2       0.00      0.00      0.00        12
     Class 3       0.00      0.00      0.00         0
     Class 4       0.00      0.00      0.00         1
     Class 5       1.00      0.14      0.24        22
     Class 6       0.67      0.67      0.67        18
     Class 7       0.81      0.55      0.66        40

    accuracy                           0.47       122
   macro avg       0.44      0.26      0.30       122
weighted avg       0.78      0.47      0.55       122


Confusion Matrix:
[[20  0  0  1  0  0  6  1]
 [ 0  0  0  0  0  0  0  1]
 [ 0 12  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  1]
 [ 0 16  2  1  0  3  0  0]
 [ 0  0  0  4  0  0 12  2]
 [ 0  8 10  0  0  0  0 22]]
[Final] Test Loss: 2.4064, Accuracy: 46.7213%, F1-Score: 0.5482
Confusion Matrix:
[[20  0

/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Epochs:  97%|█████████▋| 29/30 [02:29<00:05,  5.42s/it]


Classification Report:
              precision    recall  f1-score   support

     Class 0       1.00      0.71      0.83        28
     Class 1       0.00      0.00      0.00         1
     Class 2       0.11      0.25      0.15        12
     Class 3       0.00      0.00      0.00         0
     Class 4       0.00      0.00      0.00         1
     Class 5       1.00      0.23      0.37        22
     Class 6       0.67      0.78      0.72        18
     Class 7       0.90      0.45      0.60        40

    accuracy                           0.49       122
   macro avg       0.46      0.30      0.33       122
weighted avg       0.81      0.49      0.58       122


Confusion Matrix:
[[20  0  1  0  2  0  5  0]
 [ 0  0  0  0  0  0  1  0]
 [ 0  9  3  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  1  0]
 [ 0 10  6  0  0  5  0  1]
 [ 0  0  1  2  0  0 14  1]
 [ 0  5 17  0  0  0  0 18]]
[Final] Test Loss: 2.0540, Accuracy: 49.1803%, F1-Score: 0.5754
Confusion Matrix:
[[20  0

/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1


Classification Report:
              precision    recall  f1-score   support

     Class 0       1.00      0.54      0.70        28
     Class 1       0.00      0.00      0.00         1
     Class 2       0.08      0.25      0.12        12
     Class 3       0.00      0.00      0.00         0
     Class 4       0.00      0.00      0.00         1
     Class 5       1.00      0.18      0.31        22
     Class 6       0.65      0.72      0.68        18
     Class 7       0.78      0.45      0.57        40

    accuracy                           0.43       122
   macro avg       0.44      0.27      0.30       122
weighted avg       0.77      0.43      0.52       122


Confusion Matrix:
[[15  0  5  0  0  0  7  1]
 [ 0  0  0  0  0  0  0  1]
 [ 0  9  3  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  1]
 [ 0  7 11  0  0  4  0  0]
 [ 0  0  1  2  0  0 13  2]
 [ 0  3 19  0  0  0  0 18]]
[Final] Test Loss: 2.5452, Accuracy: 43.4426%, F1-Score: 0.5155
Confusion Matrix:
[[15  0